# Import required libraries

In [9]:
import os, io, time, sys, socket
import numpy as np
import pandas as pd

# SACLA database library and online analysis library
sys.path.append('/home/software/SACLA_tool/local/python3.5/lib/python3.5/site-packages')
import dbpy, olpy

# Import custom online library
import onlineAccess


# Plot options
import matplotlib
matplotlib.rcParams['backend']='Qt5Agg'
matplotlib.use('Qt5Agg')
%matplotlib qt5
import matplotlib.pyplot as plt
import matplotlib.widgets as widgets

# Print available detectors

In [10]:
onlineAccess.getCurrentDetectorList(3)

('MPCCD-1N0-M06-004',
 'MPCCD-2N0-1-006',
 'MPCCD-2N0-1-006-1',
 'MPCCD-2N0-1-006-2')

# How to grab a detector image

In [ ]:
NX = 512
NY = 1024
    
def dummyAvg():   
    xs = np.arange(NX)
    ys = np.arange(NY)
    XX,YY = np.meshgrid( xs,ys )
    sig = 100.
    data = np.exp( (-( XX - NX/2. )**2. - (YY-NY/2.)**2.)/(2.*sig**2.)) 
    return data
    
def dummyTile():    
    xs = np.arange(NX)
    ys = np.arange(NY)
    XX,YY = np.meshgrid( xs,ys )
    
    kx = 2*np.pi/100
    ky =  2*np.pi/100
    eps = 1.
    phi = np.random.random()*2.*np.pi
    
    data = dummyAvg() *( 1 + eps*np.cos( kx*XX+ky*YY+phi ) )
    return data

def grabDetector( detectorName, bl=3, refDet='xfel_bl_3_st_5_direct_bm_1_pd/charge' ):
    if detectorName is 'debug':
        dummyTag = onlineAccess.getNewestTag('xfel_bl_3_st_5_direct_bm_1_pd/charge' )
        dummyHightag = onlineAccess.getCurrentHighTag(3)
        return dummyTile(), dummyTag, dummyHightag
    else:
        return onlineAccess.grabNewestDetector(detectorName, bl=3, refDet='xfel_bl_3_st_5_direct_bm_1_pd/charge')
    
def thresholdDet( detector , threshold ):
    newDet = np.copy(detector)
    newDet[newDet < threshold] = 0
    return newDet

def makeRing( xc , yc , radius , NTH = 100 ):
    angles = np.linspace( 0 , 2*np.pi , NTH)
    xs = radius*np.cos(angles) + xc
    ys = radius*np.sin(angles) + yc
    return xs,ys

def plotRing(  xc , yc , radius , NTH = 100, ax=None ):
    xs,ys = makeRing( xc , yc , radius , NTH )
    ax.plot(xs , ys, 'w-', linewidth=2)
    
def findCenter( detector ):
    sumDet = np.sum(detector[:])
    
    xs = np.arange(NX)
    ys = np.arange(NY)
    XX,YY = np.meshgrid( xs,ys )
    
    xavg = np.sum( (XX*detector).flatten() ) / sumDet
    yavg = np.sum( (YY*detector).flatten() ) / sumDet
    
    return xavg, yavg
    
def plotCenter( detector , ax=None ):
    xc, yc = findCenter(detector)
    ax.scatter(xc,yc, s=100, c='w', marker='x')
    ax.set_title('Center at xc=%f and yc=%f' % (xc,yc))
    
def fastAutocorrelation( detector ):
    newDet = np.copy(detector)
    newDet -= np.mean(newDet[:])
    fftFrame = np.fft.fftshift(np.fft.fft2( newDet ))
    return np.abs(np.fft.ifftshift(np.fft.ifft2( np.abs( fftFrame )**2 )))
    
def getStatus(tag, hightag):
    machineStatusName = 'xfel_mon_bpm_bl3_0_3_beamstatus/summary'
    status = onlineAccess.getEquip( (tag,) , equip = machineStatusName , hightag=hightag )
    return (status[0]>0.1)

def  updateMPCCDOrientationPlot( ax, cax, detectorName , integrateOver = 100):
    intMPCCD = np.zeros( (NY,NX) )
    nframes = 0

    for idx in range(integrateOver):
        detFrame, tag, hightag = grabDetector(detectorName)

        if getStatus(tag,hightag) is False:
            continue

        nframes += 1.
        intMPCCD += detFrame*3.65
        time.sleep(1./29.)

    im=ax.imshow( (intMPCCD/nframes) )
    fig.colorbar(im,cax=cax,orientation='vertical')
    plotRing( NX/2. , NY/2. , radius = 60, ax=ax )
    plotRing( NX/2. , NY/2. , radius = 120, ax=ax )
    plotRing( NX/2. , NY/2. , radius = 240, ax=ax )
    plotCenter( intMPCCD, ax )
    ax.set_xlim(0,512)
    ax.set_ylim(0,1024)
    
def updateMPCCDHistogramPlot( ax, detectorName , integrateOver = 100):
    intMPCCD = np.zeros( (NY,NX) )
    nframes = 0

    for idx in range(integrateOver):
        detFrame, tag, hightag = grabDetector(detectorName)

        if getStatus(tag,hightag) is False:
            continue

        nframes += 1.
        intMPCCD += detFrame
        time.sleep(1./29.)

    histMe = np.copy(intMPCCD[:]/nframes)
    counts,bins = np.histogram(histMe,bins=100)
    db = bins[1]-bins[0]
    
    meanHeight = np.mean( histMe )
    std = np.std(histMe)
    
    ax.semilogy(bins[1:]-db/2.,counts)  
    ax.set_xlabel('Gain-corrected MPCCD pixel height')
    ax.set_ylabel('Number of pixels')
    ax.set_title('MPCCD histogram: mean=%f, std=%f' % (meanHeight,std))

# Orienting MPCCD

In [ ]:
# detectorName = 'debug'
detectorName = 'MPCCD-1N0-M06-004'

integrateOver = 10

In [ ]:
fig,ax=plt.subplots()
cax=fig.add_axes([0.8,0.1,0.1,0.6])
plt.axis('off')
fig.show()
figure_open=True


def close_event_handler(evt):
    global figure_open
    figure_open=False
    print("closed figure")
fig.canvas.mpl_connect('close_event',close_event_handler)   


while figure_open:   
    #add plots here
    ax.clear()
    cax.clear()
    updateMPCCDOrientationPlot( ax, cax, detectorName , integrateOver = integrateOver)
    
    fig.canvas.draw()
    plt.pause(2)
print("done plotting")